In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
import plotly.express as px
import plotly.graph_objects as go

# Healthy Data Run 1

In [ ]:
df_healthy1 = pd.read_csv('/content/drive/MyDrive/Thesis/HealthyData/RCS_250405_2109_5S2P_Healthy_Run1_1.csv')

In [ ]:
df_healthy1.head()

,Time,V1,V2,V3,V4,V5,IApp,DV1,DV2,DV3,DV4,DV5,DT,FaultIN
0,1.00,6.836144,6.513869,6.495558,6.837365,6.839196,-0.023823,0.322275,0.018311,-0.341807,-0.001831,0.003052,0.143398,0
1,1.05,6.836144,6.513869,6.495558,6.837365,6.839196,-0.023823,0.322275,0.018311,-0.341807,-0.001831,0.003052,0.143398,0
2,1.10,6.836144,6.513869,6.495558,6.837365,6.839196,-0.023823,0.322275,0.018311,-0.341807,-0.001831,0.003052,0.143398,0
3,1.15,6.836144,6.513869,6.495558,6.837365,6.839196,-0.023823,0.322275,0.018311,-0.341807,-0.001831,0.003052,0.143398,0
4,1.20,6.836144,6.513869,6.495558,6.837365,6.839196,-0.023823,0.322275,0.018311,-0.341807,-0.001831,0.003052,0.143398,0


In [ ]:
df_healthy1.FaultIN.value_counts()

,count
FaultIN,
0,2758363


In [ ]:
# df_healthy1["Time_min"] = df_healthy1["Time"] // 60
df_healthy1["Time_30sec"] = df_healthy1["Time"] // 30


# df_healthy1_minute_median = df_healthy1.groupby("Time_min").median().reset_index()
# df_healthy1_minute_max = df_healthy1.groupby("Time_min").max().reset_index()
df_healthy1_30sec_median = df_healthy1.groupby("Time_30sec").median().reset_index()
df_healthy1_30sec_max = df_healthy1.groupby("Time_30sec").max().reset_index()

In [ ]:
corr_matrix = df_healthy1_30sec_median.corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    labels=dict(x="Variable", y="Variable", color="Correlation"),
    title="Correlation Heatmap - 30 sec agg, median"
)

fig.update_layout(
    xaxis_tickangle=45,
    height=700,
    width=1200
)

fig.show()

In [ ]:
corr_matrix = df_healthy1_30sec_max.corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    labels=dict(x="Variable", y="Variable", color="Correlation"),
    title="Correlation Heatmap - 30 second agg, max"
)

fig.update_layout(
    xaxis_tickangle=45,
    height=700,
    width=1200
)

fig.show()

In [ ]:
variables_to_plot = [col for col in df_healthy1_30sec_median.columns if col not in ["FaultIN", "Time"]]

fig = go.Figure()

for var in variables_to_plot:
    fig.add_trace(go.Scatter(
        x=df_healthy1_30sec_median["Time"],
        y=df_healthy1_30sec_median[var],
        mode='lines',
        name=var
    ))

fault_times = df_healthy1_30sec_median[df_healthy1_30sec_median["FaultIN"] == 1]["Time"]

for ft in fault_times:
    fig.add_vline(
        x=ft,
        line=dict(color="red", width=1, dash="dash"),
        annotation_text="Fault",
        annotation_position="top right"
    )

fig.update_layout(
    title="Variables Over Time with Fault Markers - 30 sec, median",
    xaxis_title="Time (seconds)",
    yaxis_title="Value",
    legend=dict(orientation="h"),
    height=600
)

fig.show()


In [ ]:
variables_to_plot = [col for col in df_healthy1_30sec_max.columns if col not in ["FaultIN", "Time"]]

fig = go.Figure()

for var in variables_to_plot:
    fig.add_trace(go.Scatter(
        x=df_healthy1_30sec_max["Time"],
        y=df_healthy1_30sec_max[var],
        mode='lines',
        name=var
    ))

fault_times = df_healthy1_30sec_max[df_healthy1_30sec_max["FaultIN"] == 1]["Time"]

for ft in fault_times:
    fig.add_vline(
        x=ft,
        line=dict(color="red", width=1, dash="dash"),
        annotation_text="Fault",
        annotation_position="top right"
    )

fig.update_layout(
    title="Variables Over Time with Fault Markers - 30 sec, max",
    xaxis_title="Time (seconds)",
    yaxis_title="Value",
    legend=dict(orientation="h"),
    height=600
)

fig.show()


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import plotly.graph_objects as go

def shannon_entropy(series, bins=10):
    hist, _ = np.histogram(series, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist, base=2)

df = df_healthy1_30sec_median.copy()

window_size = 10
entropy_values = [
    shannon_entropy(df["DT"].iloc[i : i + window_size])
    for i in range(len(df) - window_size + 1)
]

time_vals = df["Time"].iloc[: len(entropy_values)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=time_vals,
    y=entropy_values,
    mode="lines",
    name="Shannon Entropy (DT)",
    line=dict(color="red")
))

fig.update_layout(
    title="Shannon Entropy of DT Over Time (30s median)",
    xaxis_title="Time (seconds)",
    yaxis_title="Entropy",
    height=500
)

fig.show()


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import plotly.graph_objects as go

def shannon_entropy(series, bins=10):
    hist, _ = np.histogram(series, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist, base=2)

df = df_healthy1_30sec_max.copy()

window_size = 10
entropy_values = [
    shannon_entropy(df["DT"].iloc[i : i + window_size])
    for i in range(len(df) - window_size + 1)
]

time_vals = df["Time"].iloc[: len(entropy_values)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=time_vals,
    y=entropy_values,
    mode="lines",
    name="Shannon Entropy (DT)",
    line=dict(color="red")
))

fig.update_layout(
    title="Shannon Entropy of DT Over Time (30s median)",
    xaxis_title="Time (seconds)",
    yaxis_title="Entropy",
    height=500
)

fig.show()


# Healthy Data Run 2

In [ ]:
df_healthy2 = pd.read_csv('/content/drive/MyDrive/Thesis/HealthyData/RCS_250410_1021_5S2P_Healthy_Run2_1.csv')

In [ ]:
df_healthy2.head()

,Time,V1,V2,V3,V4,V5,IApp,DV1,DV2,DV3,DV4,DV5,DT,FaultIN
0,1.00,7.641222,7.640001,7.624742,7.643358,7.645926,-0.027485,0.001221,0.015259,-0.018616,-0.002568,0.004704,0.024464,0
1,1.05,7.641222,7.640001,7.624742,7.643358,7.645674,-0.027485,0.001221,0.015259,-0.018616,-0.002316,0.004452,0.017437,0
2,1.10,7.641222,7.640001,7.624742,7.643358,7.645926,-0.027485,0.001221,0.015259,-0.018616,-0.002568,0.004704,0.022317,0
3,1.15,7.641222,7.640001,7.624742,7.643358,7.645674,-0.027485,0.001221,0.015259,-0.018616,-0.002316,0.004452,0.024464,0
4,1.20,7.641222,7.640001,7.624742,7.643358,7.645926,-0.027485,0.001221,0.015259,-0.018616,-0.002568,0.004704,0.026806,0


In [ ]:
df_healthy1.FaultIN.value_counts()

,count
FaultIN,
0,2758363


In [ ]:
# df_healthy2["Time_min"] = df_healthy2["Time"] // 60
df_healthy2["Time_30sec"] = df_healthy2["Time"] // 30


# df_healthy2_minute_median = df_healthy2.groupby("Time_min").median().reset_index()
# df_healthy2_minute_max = df_healthy2.groupby("Time_min").max().reset_index()
df_healthy2_30sec_median = df_healthy2.groupby("Time_30sec").median().reset_index()
df_healthy2_30sec_max = df_healthy2.groupby("Time_30sec").max().reset_index()

In [ ]:
corr_matrix = df_healthy2_30sec_median.corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    labels=dict(x="Variable", y="Variable", color="Correlation"),
    title="Correlation Heatmap - 30 second agg, median"
)

fig.update_layout(
    xaxis_tickangle=45,
    height=700,
    width=1200
)

fig.show()

In [ ]:
corr_matrix = df_healthy2_30sec_max.corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    labels=dict(x="Variable", y="Variable", color="Correlation"),
    title="Correlation Heatmap - 30 second agg, max"
)

fig.update_layout(
    xaxis_tickangle=45,
    height=700,
    width=1200
)

fig.show()

In [ ]:
variables_to_plot = [col for col in df_healthy2_30sec_median.columns if col not in ["FaultIN", "Time"]]

fig = go.Figure()

for var in variables_to_plot:
    fig.add_trace(go.Scatter(
        x=df_healthy2_30sec_median["Time"],
        y=df_healthy2_30sec_median[var],
        mode='lines',
        name=var
    ))

fault_times = df_healthy2_30sec_median[df_healthy2_30sec_median["FaultIN"] == 1]["Time"]

for ft in fault_times:
    fig.add_vline(
        x=ft,
        line=dict(color="red", width=1, dash="dash"),
        annotation_text="Fault",
        annotation_position="top right"
    )

fig.update_layout(
    title="Variables Over Time with Fault Markers - 30 sec, median",
    xaxis_title="Time (seconds)",
    yaxis_title="Value",
    legend=dict(orientation="h"),
    height=600
)

fig.show()


In [ ]:
variables_to_plot = [col for col in df_healthy2_30sec_max.columns if col not in ["FaultIN", "Time"]]

fig = go.Figure()

for var in variables_to_plot:
    fig.add_trace(go.Scatter(
        x=df_healthy2_30sec_max["Time"],
        y=df_healthy2_30sec_max[var],
        mode='lines',
        name=var
    ))

fault_times = df_healthy2_30sec_max[df_healthy2_30sec_max["FaultIN"] == 1]["Time"]

for ft in fault_times:
    fig.add_vline(
        x=ft,
        line=dict(color="red", width=1, dash="dash"),
        annotation_text="Fault",
        annotation_position="top right"
    )

fig.update_layout(
    title="Variables Over Time with Fault Markers - 30 sec, max",
    xaxis_title="Time (seconds)",
    yaxis_title="Value",
    legend=dict(orientation="h"),
    height=600
)

fig.show()


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import plotly.graph_objects as go

def shannon_entropy(series, bins=10):
    hist, _ = np.histogram(series, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist, base=2)

df = df_healthy2_30sec_median.copy()

window_size = 10
entropy_values = [
    shannon_entropy(df["DT"].iloc[i : i + window_size])
    for i in range(len(df) - window_size + 1)
]

time_vals = df["Time"].iloc[: len(entropy_values)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=time_vals,
    y=entropy_values,
    mode="lines",
    name="Shannon Entropy (DT)",
    line=dict(color="red")
))

fig.update_layout(
    title="Shannon Entropy of DT Over Time (30s median)",
    xaxis_title="Time (seconds)",
    yaxis_title="Entropy",
    height=500
)

fig.show()


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import plotly.graph_objects as go

def shannon_entropy(series, bins=10):
    hist, _ = np.histogram(series, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist, base=2)

df = df_healthy2_30sec_max.copy()

window_size = 10
entropy_values = [
    shannon_entropy(df["DT"].iloc[i : i + window_size])
    for i in range(len(df) - window_size + 1)
]

time_vals = df["Time"].iloc[: len(entropy_values)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=time_vals,
    y=entropy_values,
    mode="lines",
    name="Shannon Entropy (DT)",
    line=dict(color="red")
))

fig.update_layout(
    title="Shannon Entropy of DT Over Time (30s max)",
    xaxis_title="Time (seconds)",
    yaxis_title="Entropy",
    height=500
)

fig.show()
